## Dose-Response Plotter

In [1]:
CC=0.1

We call the libraries and the data from the experiment

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import os

if not os.path.exists("Figures"):
    os.makedirs("Figures")

In [3]:
def load_data(filedir):
    
    mydata = np.genfromtxt(filedir,delimiter="\t")
    TotalDensity = [i[0] for i in mydata] #Total bacterial density
    Suc = [i[1] for i in mydata] #Susceptible strain total density
    Res = [i[2] for i in mydata] #Resistant strain total density
    Rel_Freq = [i[2]/(i[1]+i[2]) for i in mydata] #Resistant Relative Frequency
    Suc_win = [i[3] for i in mydata] #Number of wins of Susceptible strain
    Res_win = [i[4] for i in mydata] #Number of wins of Resistant strain
    R_S_winratio = [i[4]/(i[3]+i[4]) for i in mydata]#node-to-node Resistant WinRatio
    Netdensity = [i[6] for i in mydata] #Network density
    clustc = [i[7] for i in mydata] #Network clustering coefficient
    clo_centr = [i[8] for i in mydata] #Closeness centrality
    ant_deg = [i[9] for i in mydata] #Antibiotic source connectivity
    bet_centr = [i[10] for i in mydata] #Betweeness centrality
    ant_con = [i[11] for i in mydata]
    ME = [i[12] for i in mydata]
    
    d = {'TotDen':TotalDensity,'SucDen':Suc,'ResDen':Res,'Rel_Freq':Rel_Freq,
           'SucWin':Suc_win, 'ResWin':Res_win, 'R_S_winratio':R_S_winratio,
           'NetDen':Netdensity,'ClustC':clustc,'CloCentr':clo_centr,'AntDeg':ant_deg,
           'BetCentr':bet_centr,'AntCon':ant_con,'ME':ME}
    return pd.DataFrame(d)

In [4]:
NamesDict = {'TotDen':'Total Bacterial Density','SucDen': 'Susceptible Bacteria Density',
             'ResDen': 'Resistant Bacteria Density','Rel_Freq': 'Resistant Relative Frequency',
       'SucWin': 'Susceptible D-Score', 'ResWin':'Resistant D-Score', 'R_S_winratio':'Ratio of winners R/S',
       'NetDen':'Network Density','ClustC':'Global Clustering Coefficient','CloCentr':'Closeness Centrality',
        'ant_deg':'Connectivity of Antibiotic Source',
       'BetCentr':'Betweenness Centrality','AntCon':'Antibiotic Concentration'}

In [5]:
sim_size=6
df_vec = [load_data("DR_N_50_meanedg_"+str(i)+"_watts_out") for i in range(sim_size)]

OSError: DR_N_50_meanedg_0_watts_out not found.

We generate the function for plotting many comparisons

In [ ]:
def plot_Dose_on_col(column,loc='best'):
    mean_matrix = [df_vec[i].groupby(['AntCon'], as_index=False).mean() for i in range(sim_size)]
    stderr_matrix = [df_vec[i].groupby(['AntCon'], as_index=False).std() for i in range(sim_size)]
    Ant_Con = mean_matrix[0]['AntCon']##########################################
    for i in range(sim_size):
        plt.plot(Ant_Con, mean_matrix[i][column])
        plt.fill_between(Ant_Con, mean_matrix[i][column]-stderr_matrix[i][column], mean_matrix[i][column]+stderr_matrix[i][column], alpha=0.5)
    
    plt.title("Dose effect on "+NamesDict[column]+" when increasing Mean Edges",y=1.08)
    plt.legend([str(i*2) for i in range(sim_size)],loc=loc,title="Mean Edges")
    plt.ylabel(NamesDict[column])
    plt.xlabel(NamesDict['AntCon'])
    
    plt.show()

In [ ]:
plot_Dose_on_col('Rel_Freq')

In [ ]:
plot_Dose_on_col('TotDen')

In [ ]:
plot_Dose_on_col('ResWin')

In [ ]:
plot_Dose_on_col('SucWin',loc='lower right')

## Analysis of Relative Frequency AND Density

In [ ]:
fontsize=15
sim_size = 6
f, ax_vec = plt.subplots(sim_size, sharex=True, sharey=False, figsize=(10,10))

mean_matrix = [df_vec[i].groupby(['AntCon'], as_index=False).mean() for i in range(sim_size)]

for i in range(sim_size):
    ax = ax_vec[i]
    ax.stackplot(range(len(mean_matrix[i])),mean_matrix[i].ResDen,mean_matrix[i].SucDen,colors=['red','blue'])
    

    ax.annotate(r'$M=%d$' % ((i+1)*2), xy=(1.03,0.5), rotation=90,
                        ha='center',va='center',xycoords='axes fraction', fontsize=fontsize)
    if i==0:
        ax.set_title('Dose-Response when increasing network connectivity (avg)',y=1.15,fontsize=fontsize)
    else:
        ax.yaxis.get_offset_text().set_visible(False)
        if i==sim_size-1:
            ax.legend(['Resistant','Susceptible'])
    
f.subplots_adjust(wspace=0,hspace=0)
plt.xlabel("Antibiotic Concentration",fontsize=fontsize)
f.text(0.05, 0.5, 'Bacterial Population Density', va='center', rotation='vertical',fontsize = fontsize)

plt.savefig("Figures/DR_varME_2to8_CC+"+str(CC)+".png")
plt.show()

In [ ]:
def get_DScore_ICs(mean_matrix,column,percentage):
    if column=='ResWin':
        maxVal = np.max(mean_matrix[0][column]+mean_matrix[0]['SucWin'])
        return[mean_matrix[i][(mean_matrix[i][column]+mean_matrix[i]['SucWin'] < maxVal*percentage/100)].iloc[0].AntCon.round(1) for i in range(2,sim_size)] #Omiting the first cases when the IC won't exist. 

    else:
        maxVal = np.max([mean_matrix[0][column] for i in range(sim_size)])
    return[mean_matrix[i][(mean_matrix[i][column] < maxVal*percentage/100)].iloc[0].AntCon.round(1) for i in range(1,sim_size)] #Omiting the first cases when the IC won't exist. 
    

In [ ]:
get_DScore_ICs(mean_matrix,'TotDen',75)

In [ ]:
notplotted=2
ICs = range(10,90)
ICCons = [get_DScore_ICs(mean_matrix,'ResWin',i) for i in ICs]

final_sim_size = sim_size-notplotted
for i in range(final_sim_size):
    plt.plot(ICs,ICCons)
plt.legend([str((i+notplotted)*2) for i in range(final_sim_size)],title="ME")
plt.show()

In [ ]:
fontsize=15
sim_size = 5

f, ax_vec = plt.subplots(sim_size, sharex=True, sharey=False, figsize=(10,10))

mean_matrix = [df_vec[i].groupby(['AntCon'], as_index=False).mean() for i in range(sim_size)]

DScore_IC50 = [mean_matrix[i].TotDen for i in range(len(mean_matrix[i]))]

for i in range(sim_size):
    ax = ax_vec[i]
    ax.stackplot(range(len(mean_matrix[i])),mean_matrix[i].ResWin,mean_matrix[i].SucWin,colors=['red','blue'])
    

    ax.annotate(r'$M=%d$' % ((i+1)*2), xy=(1.03,0.5), rotation=90,
                        ha='center',va='center',xycoords='axes fraction', fontsize=fontsize)
    if i==0:
        ax.set_title('Dose-Response when increasing network connectivity (means)',y=1.15,fontsize=fontsize)
    else:
        ax.yaxis.get_offset_text().set_visible(False)
        if i==sim_size-2:
            ax.legend(['Resistant','Susceptible'])
    
f.subplots_adjust(wspace=0,hspace=0)
plt.xlabel("Antibiotic Concentration",fontsize=fontsize)
f.text(0.05, 0.5, 'Bacterial Population Density', va='center', rotation='vertical',fontsize = fontsize)

plt.savefig("Figures/DR_varME_2to8_CC+"+str(CC)+".png")
plt.show()